In [2]:
#%matplotlib widget

import healpy as hp
import numpy as np
import matplotlib.pyplot as plt

import circle
import plot_utils
import utils




### Functions for linear parameterization


In [ ]:
#### check that the ray is not in the plane defined by any of the healpix edges
def direction_star_to_ISRF_point(isrf_pos, star_pos):
    """
    Calculate the direction from the star to the ISRF point.
    """
    direction = isrf_pos - star_pos
    direction_abs = np.dot(direction, direction) ** 0.5
    star_dot_direction = np.dot(star_pos, direction)
    return direction, direction_abs, star_dot_direction

def ray_linear_parameterization(star_pos,direction, t):
    """ This function returns the position of a point on the ray defined by star_pos and direction at parameter t.
    star_pos is the position of the star, direction is the direction of the ray, and t is the parameter.
    The function returns the position of the point on the ray at parameter t.
    The ray is defined as the set of points {star_pos + t*direction | t in R}.
    star_pos is a 3D cartesian vector, direction is a 3D vector, and t is a scalar.
    
    """
    return star_pos + t*direction


def p(star_pos,direction, t):
    """    This function returns the distance from the star to the point on the ray defined by star_pos and direction at parameter t.
    star_pos is the position of the star, direction is the direction of the ray, and t is the parameter.
    The function returns the distance from the star to the point on the ray at parameter t.
    The ray is defined as the set of points {star_pos + t*direction | t in R}.
    star_pos is a 3D cartesian vector, direction is a 3D vector, and t is a scalar.
    The distance is defined as the euclidean distance from the star to the point on the ray at parameter t.
    The function returns a scalar.
    """
    x0,y0,z0 = star_pos
    a,b,c = direction
    p = ((x0 + t*a)**2 + (y0 + t*b)**2 + (z0 + t*c)**2)**0.5  
    return p

def find_crossing_positions(star_r,star_dot_direction,direction_abs,d):
    """ 
    
    This function calculates the crossing positions of a ray defined by star_r and star_dot_direction
    with a sphere of radius d. It returns the two possible intersection parameters t1 and t2.
    """
    a = direction_abs**2
    b = 2*star_dot_direction
    c = star_r**2-d**2
    t1 = (-b + (b**2 - 4*a*c)**0.5)/(2*a)
    t2 = (-b - (b**2 - 4*a*c)**0.5)/(2*a)
    return t1,t2


find_crossing_positions(star_r,star_dot_direction,direction_abs,0.5)

(1.5, 0.5)

### Functions for the circles and arches implementation

In [6]:

# test_example.test_circle_implementation()
# test_example.test_arches_implementation()


In [ ]:
https://healpy.readthedocs.io/en/latest/generated/healpy.query_polygon.html

In [ ]:
vertices = np.array([[1,0,0],[1,1.1,0],[1,0,1.1]])

hp.query_polygon(nside=1,vertices=vertices, inclusive=True)

In [ ]:

N_side = 16  # Choose your resolution
pixel_id = 42  # Example pixel index

# Get the boundaries of the pixel
boundaries = hp.boundaries(N_side, pixel_id, step=1)
theta, phi = hp.vec2ang(np.transpose(boundaries))

# Convert to degrees for plotting
theta_deg = np.degrees(theta)
phi_deg = np.degrees(phi)

# Plot
# plt.figure()
hp.visufunc.projscatter(theta, phi, lonlat=False, c='red')  # Plot edges
hp.visufunc.projtext(np.mean(theta), np.mean(phi), str(pixel_id), lonlat=False, color='blue')  # Center
plt.show()
reddening_distance_slices = np.array([0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0])

isrf_point_l = 0
isrf_point_b = 0
isrf_point_r = 0

star_l = 1
star_b = 1
star_r = 1



star_pos = utils.galactic_to_cartesian(star_l,star_b,star_r)
isrf_pos = utils.galactic_to_cartesian(isrf_point_l,isrf_point_b,isrf_point_r)

# Calculate the direction from the star to the ISRF point
direction, direction_abs, star_dot_direction = direction_star_to_ISRF_point(isrf_pos, star_pos)


        
